In [172]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.model_selection import train_test_split

In [173]:
employment_gdp_dataframe = pd.read_excel("./HW2_Datasets.xlsx")
employment_gdp_dataframe.head()

,Quarter,Employment (millions),GDP (billions)
0,Q1 2020,155,21500
1,Q2 2020,152,21200
2,Q3 2020,150,21700
3,Q4 2020,153,22000
4,Q1 2021,154,22200


In [174]:
employment_gdp_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Quarter                20 non-null     object
 1   Employment (millions)  20 non-null     int64 
 2   GDP (billions)         20 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 612.0+ bytes


In [175]:
employment_gdp_dataframe.describe()

,Employment (millions),GDP (billions)
count,20.000000,20.000000
mean,164.150000,23437.500000
std,9.669676,1385.818683
min,150.000000,21200.000000
25%,155.750000,22350.000000
50%,163.000000,23400.000000
75%,171.500000,24550.000000
max,181.000000,25750.000000


In [176]:
employment_gdp_dataframe["Year"] = employment_gdp_dataframe["Quarter"].str[-4:].astype(int)
employment_gdp_dataframe['Quarter_num'] = employment_gdp_dataframe['Quarter'].str[1].astype(int)
employment_gdp_dataframe["Lagged Employment"] = employment_gdp_dataframe["Employment (millions)"].shift(1)
employment_gdp_dataframe = employment_gdp_dataframe.dropna()
employment_gdp_dataframe = employment_gdp_dataframe.sort_values(by= ["Year", "Quarter_num"]).reset_index().drop(columns= "index")#.rename(columns={'index': 'Coded Time'})
employment_gdp_dataframe = employment_gdp_dataframe.reset_index().rename(columns={'index': 'Coded Time'})

In [177]:
#OnehotEncoding
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse_output= False, drop= "first")

encoded_quarters = encoder.fit_transform(employment_gdp_dataframe[["Quarter_num"]]).astype(int)
df_encoded_quarters = pd.DataFrame(data= encoded_quarters, columns= encoder.get_feature_names_out())
print(df_encoded_quarters.head())
print(encoder.feature_names_in_)
df_encoded = pd.concat([employment_gdp_dataframe, df_encoded_quarters], axis= 1)
df_encoded.head()

   Quarter_num_2  Quarter_num_3  Quarter_num_4
0              1              0              0
1              0              1              0
2              0              0              1
3              0              0              0
4              1              0              0
['Quarter_num']


,Coded Time,Quarter,Employment (millions),GDP (billions),Year,Quarter_num,Lagged Employment,Quarter_num_2,Quarter_num_3,Quarter_num_4
0,0,Q2 2020,152,21200,2020,2,155.0,1,0,0
1,1,Q3 2020,150,21700,2020,3,152.0,0,1,0
2,2,Q4 2020,153,22000,2020,4,150.0,0,0,1
3,3,Q1 2021,154,22200,2021,1,153.0,0,0,0
4,4,Q2 2021,156,22400,2021,2,154.0,1,0,0


In [178]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 10 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Coded Time             19 non-null     int64  
 1   Quarter                19 non-null     object 
 2   Employment (millions)  19 non-null     int64  
 3   GDP (billions)         19 non-null     int64  
 4   Year                   19 non-null     int64  
 5   Quarter_num            19 non-null     int64  
 6   Lagged Employment      19 non-null     float64
 7   Quarter_num_2          19 non-null     int64  
 8   Quarter_num_3          19 non-null     int64  
 9   Quarter_num_4          19 non-null     int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 1.6+ KB


In [ ]:
# df_encoded["GDP (billions)"] = df_encoded["GDP (billions)"].astype(float)
# df_encoded["Employment (millions)"] =  df_encoded["Employment (millions)"].astype(float)
# df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Quarter                20 non-null     object 
 1   Employment (millions)  20 non-null     float64
 2   GDP (billions)         20 non-null     float64
 3   Year                   20 non-null     int64  
 4   Quarter_num            20 non-null     int64  
 5   Quarter_num_2          20 non-null     int64  
 6   Quarter_num_3          20 non-null     int64  
 7   Quarter_num_4          20 non-null     int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 1.4+ KB


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(df_encoded.drop(columns= ["Quarter", "Year", "GDP (billions)", "Employment (millions)"]), employment_gdp_dataframe[["GDP (billions)"]], test_size=0.2, random_state= 42)

In [179]:
X_train = df_encoded.drop(columns= ["Quarter", "Year", "GDP (billions)", "Employment (millions)", "Quarter_num"])
y_train = df_encoded[["GDP (billions)"]]

In [180]:
X_train_with_const = sm.add_constant(X_train)
# X_test_with_const = sm.add_constant(X_test)

In [181]:
X_train_with_const

,const,Coded Time,Lagged Employment,Quarter_num_2,Quarter_num_3,Quarter_num_4
0,1.0,0,155.0,1,0,0
1,1.0,1,152.0,0,1,0
2,1.0,2,150.0,0,0,1
3,1.0,3,153.0,0,0,0
4,1.0,4,154.0,1,0,0
5,1.0,5,156.0,0,1,0
6,1.0,6,158.0,0,0,1
7,1.0,7,159.0,0,0,0
8,1.0,8,160.0,1,0,0
9,1.0,9,162.0,0,1,0


In [182]:
y_train.head()

,GDP (billions)
0,21200
1,21700
2,22000
3,22200
4,22400


In [183]:
print(type(X_train_with_const))
print(type(y_train))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [184]:
ols_model = sm.OLS(y_train, X_train_with_const).fit()

In [185]:
y_predicted = ols_model.predict(X_train_with_const)

In [186]:
ols_model.summary()

/home/buddha-thapa-magar/anaconda3/lib/python3.12/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=19
  res = hypotest_fun_out(*samples, **kwds)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         GDP (billions)   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     1501.
Date:                Sat, 09 Nov 2024   Prob (F-statistic):           1.80e-17
Time:                        15:17:20   Log-Likelihood:                -102.90
No. Observations:                  19   AIC:                             217.8
Df Residuals:                      13   BIC:                             223.5
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const              2.427e+04   1356.550     17.890      0.000    2.13e+04    2.72e+04
Coded Time          268.7975     14.781     18.186      0.000     236.866     300.729
Lagged Employment   -19.3557      9.134     -2.119      0.054     -39.089       0.377
Quarter_num_2       -15.4770     45.649     -0.339      0.740    -114.097      83.143
Quarter_num_3        35.0812     44.534      0.788      0.445     -61.128     131.290
Quarter_num_4        25.6395     44.227      0.580      0.572     -69.907     121.186
==============================================================================
Omnibus:                        3.870   Durbin-Watson:                   0.790
Prob(Omnibus):                  0.144   Jarque-Bera (JB):                1.443
Skew:                           0.194   Prob(JB):                        0.486
Kurtosis:                       1.707   Cond. No.                     1.47e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.47e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [187]:
ols_model.resid

0    -52.852423
1     69.724670
2     71.657489
3     86.566630
4     52.601872
5    -28.042401
6    -48.686674
7    -72.488987
8     -6.453744
9    -87.098018
10   -57.742291
11   -12.188877
12   -26.797907
13    -7.442181
14   -47.442181
15    -1.888767
16    33.502203
17    52.857930
18    82.213656
dtype: float64

In [188]:
from sklearn.metrics import mean_squared_error, r2_score

In [189]:
mean_squared_error(y_true= y_train, y_pred= y_predicted)

2961.8739856248053

Residual Scatterplot: Create a scatterplot of the residuals against the lagged employment. Does the plot suggest that the residuals are homoscedastic?